In [22]:
results_estimation="variable-results/q1_64"
kind=False
g5k=True
# infra
job_managers_qty = 2
kafka_qty = 0
kafka_partitions=32
kafka_replicas=8
kafka_node_selector="jobmanager"
reset_kafka_data = False
filter_data = ""
task_managers_qty = 16
source_parallelism = 32

datagen_configuration = {
    "parallelism": 16,
    "cpu": 8,
    "memory" : 16384,
    "task_managers_qty": 2,
    "task_slots": 8,
    "timeout": 600,
    "notebook": "/xp_datagen"
}

datagen_configuration = None 
results_path = "verification-results/"

notebook = "/xp_intro_q1_datagen"
warmup = 120
sensitivity = 0.01
ratio = 1
timeout = 600
xp_name = "query"

In [15]:
#papermill_description=Initialize libraries
import sys
import os
from pathlib import Path 
from time import sleep
import logging 
%run ../common/scripts.ipynb
if "../streambed" not in sys.path:
    sys.path.append("../streambed")
import streambed
streambed.setup_logging(default_path="./logging.yaml", default_level=logging.WARN)
logger = logging.getLogger('streambed')
if kind:
    streambed.init_kind()
if g5k:
    streambed.init_g5k()

logging.info("Execute notebook")

In [23]:
#papermill_description=Reset_Kafka
if reset_kafka_data:
    #papermill_description=Initialize libraries
    init_label_nodes(jobmanagers_qty=job_managers_qty, kafka_qty=kafka_qty)

    address = "127-0-0-1"
    port=30081

    !kubectl delete -n kafka kafka/my-cluster
    sleep(10)
    #run_command("kubectl create -f ./kafka.yaml", shell=False)
    streambed.deploy_kafka(kafka_partitions, kafka_replicas, node_selector=kafka_node_selector)
    run_command("kubectl wait kafka/my-cluster --for=condition=Ready --timeout=300s -n kafka", shell=False)
    # Kafka UI
    run_command("helm install -f values-kowl.yaml -n kafka kowl cloudhut/kowl", shell=False)
    #(ip_url, dns_url) = get_service_public_address("kafka", "manager", "kowl", 80)
    (manager_node, jobmanager_node, taskmanager_node) = get_label_nodes()
    print("Kafka Kowl: {} - Ingress: {}".format(manager_node, "http://kowl.{}.sslip.io:{}".format(address,port)))    
    !kubectl apply -f ./kafka-bridge.yaml
    !kubectl apply -f ./kafka-bridge-service.yaml

In [24]:
#papermill_description=Load_estim_data
df, parallelism = streambed.read_results(results_estimation + ".csv")
if filter_data != "":
    df = df.query(filter_data)
maxdf = df.loc[df.groupby(["used_task_slots", "memory"])["observed_source_rate"].idxmax()]
display(df[["used_task_slots", "memory", "observed_source_rate"]])
logger.debug(df[["used_task_slots", "memory", "observed_source_rate"]])
list_xp = []
for i in range(len(maxdf.index)):
    run_dict = maxdf.iloc[i].to_dict()
    if isinstance(run_dict["task_parallelism"], str) == False: 
        continue
    observed_source_rate = run_dict["observed_source_rate"]
    run_dict["throughputs"] = [int(observed_source_rate * ratio), int((ratio * observed_source_rate)*1.2), int((ratio * observed_source_rate)*1.5)]
    run_dict["notebook"] = notebook
    if datagen_configuration is not None:
        run_dict["timeout"] = datagen_configuration["timeout"]
    run_dict["warmup"] = warmup
    run_dict["sensitivity"] = sensitivity # if >sensibility => pass

    run_dict["task_managers_qty"] = task_managers_qty
    run_dict["source_parallelism"] = source_parallelism # count about 
    run_dict["timeout"] = timeout
    run_dict["file"] = results_estimation
    list_xp.append(run_dict)
    

In [25]:
#papermill_description=Loop_verification
import datetime            
now = datetime.datetime.now()
now_str = now.strftime("%Y%m%d%H%M%S")     
path= "{}/{}-{}".format(results_path, xp_name, now_str)

streambed.loop_verification(list_xp, datagen_configuration, results_path=path, detail_metrics=True)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

IndexError: list index out of range